### Python packages 
Installing prerequisites: langchain and langgraph libraries

In [1]:
!pip install --quiet langchain langchain-community langchain-openai


[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


### Configure LLM

Always run this, before trying out anything else. 

You can use OpenAI or AzureOpenAI. 

ALTERNATIVE: Using AzureOpenAI instance as LLM

In [ ]:
AZURE_OPENAI_ENDPOINT = AZURE_OPENAI_EMBEDDING_ENDPOINT = ""
AZURE_OPENAI_API_KEY = ""
AZURE_OPENAI_API_VERSION = "2025-01-01-preview"
AZURE_OPENAI_DEPLOYMENT_NAME = "gpt-4o-mini"

from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    api_key=AZURE_OPENAI_API_KEY,
    api_version=AZURE_OPENAI_API_VERSION,
    deployment_name=AZURE_OPENAI_DEPLOYMENT_NAME,
)

ALTERNATIVE: Using OpenAI as LLM

In [ ]:
OPENAI_API_KEY = ""

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=OPENAI_API_KEY, model="gpt-4o")

## Chapter 1 Tools, agentic workflows

Scenario is a travel agent, who can manage travels (hotels, flights, car rentals). 

This is NOT a true agent prompt yet, only a chatbot that can execute tools. 

### Create mock tools

Ref: https://python.langchain.com/v0.2/docs/how_to/tool_calling/

In [3]:
from langchain.tools import tool

# Non-sensitive tools (GET requests)

@tool
def mock_query_hotel_bookings(user: str) -> str:
    """This tool can be used to fetch information about hotel bookings."""

    return "Current bookings: 2 rooms booked for 3 nights each in Budapest."

@tool
def mock_query_flight_bookins(user: str) -> str:
    """This tool can be used to fetch information about flight bookings."""
    return "Current bookings: 2 flights booked to Budapest from Berlin."

@tool
def mock_query_car_rentals(user: str) -> str:
    """This tool can be used to fetch information about car rentals."""
    return "Current bookings: 1 car rented for 3 days in Budapest."

# Sensitive tools (e.g. POST requests)

@tool
def mock_book_flight(user: str, origin: str, destination: str, date: str) -> str:
    """This sensitive tool can be used to book flights."""
    return f"Flight booked from {origin} to {destination} on {date}. Flight number is LH123 with Lufthansa."

@tool
def mock_book_hotel(user: str, city: str, checkin: str, checkout: str) -> str:
    """This sensitive tool can be used to book hotels."""
    return f"Hotel booked in {city} from {checkin} to {checkout}. Room number is 123."

@tool
def mock_book_car(user: str, city: str, date: str) -> str:
    """This sensitive tool can be used to book cars."""
    return f"Car rented in {city} on {date}. Car model is BMW 3 Series."

### Run a simple tool selector chain

Ref. https://python.langchain.com/v0.1/docs/modules/model_io/chat/function_calling/

In [4]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, MessagesPlaceholder
from langchain_core.messages import SystemMessage

TOOL_SELECTOR_PROMPT_RAW = """
    You are part of a chat assistant that helps users with their travels.
    If none of the available tools are useful, inform the user that you are NOT able to help.

    Important: Always inform the user what is going to be done, especially during tool calls.
    Always verify if you have all the necessary information to use the tool, otherwise ask the user first for the missing information.
    You need to ask for user permission to use sensitive tools.

    If the tool to be used is sensitive (as mentioned in its description), start your response with "sensitive:" in this case.
    HOWEVER, only add "sensitive:" to response content when the exact tool is to be called, not during discussion or follow up questions or while asking for confirmation. 
    Also, don't include "sensitive:" in the response content if the tool to be called is not sensitive.
    You cannot leave input parameters empty  or invalidt values for tool calls!! 
    """

tool_selector_prompt =  ChatPromptTemplate.from_messages([
    SystemMessage(content=TOOL_SELECTOR_PROMPT_RAW), 
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
]
)

plain_tools = [mock_query_hotel_bookings, mock_query_flight_bookins, mock_query_car_rentals]
sensitive_tools = [mock_book_flight, mock_book_hotel, mock_book_car]
all_tools = plain_tools + sensitive_tools
llm = llm.bind_tools(all_tools)

tool_chain = tool_selector_prompt | llm
selector_response = tool_chain.invoke({"input": "What flights do I have booked?", "history" : []})
selector_response.pretty_print()

================================== Ai Message ==================================

Could you please provide me with your username or the name associated with your flight bookings? This information is necessary to retrieve your flight details.


Let's see how sensitive tools are called. 

You should also play with the system prompt. See what happens if you remove lines/instructions, etc.

In [5]:
selector_response = tool_chain.invoke({"input": "Book me a flight from Helsinki to Budapest on 2023-01-01. My username is hegeduscs. Proceed with the booking.", "history" : []})
selector_response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  mock_book_flight (call_YM6WAVRw9ATpbn9e2SEeFFgB)
 Call ID: call_YM6WAVRw9ATpbn9e2SEeFFgB
  Args:
    user: hegeduscs
    origin: Helsinki
    destination: Budapest
    date: 2023-01-01


### Add chat memory and test a simple agent flow

Let's see how we can create a conversational chatbot with these tools. 

In [6]:
from langchain_community.chat_message_histories import SQLChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

def get_session_history(session_id):
    return SQLChatMessageHistory(session_id, connection="sqlite:///tool_agent_memory.db")

tool_chain_with_history = RunnableWithMessageHistory(
    tool_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="history",
)

tool_chain_with_history.get_session_history("1").clear()

Ok, now let's see some actual chatting.

In [7]:
selector_response = tool_chain_with_history.invoke({"input": "Book me a flight to Budapest on 2023-01-01."}, config={"configurable": {"session_id": "1"}},)
selector_response.pretty_print()

================================== Ai Message ==================================

I need to know your departure location to book the flight for you. Could you please provide the city or airport you will be flying from?


In [8]:
selector_response = tool_chain_with_history.invoke({"input": "hegeduscs is the username and London is the departure city"}, config={"configurable": {"session_id": "1"}},)
selector_response.pretty_print()

================================== Ai Message ==================================

Just to confirm, you would like to book a flight from London to Budapest on January 1, 2023. I will proceed with the booking. 

Please confirm so I can finalize the booking.


In [9]:
selector_response = tool_chain_with_history.invoke({"input": "I confirm the booking"}, config={"configurable": {"session_id": "1"}},)
selector_response.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  mock_book_flight (call_VDqEiMGqSfjLdBgPLHqTFjFg)
 Call ID: call_VDqEiMGqSfjLdBgPLHqTFjFg
  Args:
    user: hegeduscs
    origin: London
    destination: Budapest
    date: 2023-01-01


Let's see full chat history so far. 

In [10]:
history = tool_chain_with_history.get_session_history("1").get_messages()
history_prompt = ChatPromptTemplate.from_messages(history)
history_prompt.pretty_print()

================================ Human Message =================================

Book me a flight to Budapest on 2023-01-01.

================================== Ai Message ==================================

I need to know your departure location to book the flight for you. Could you please provide the city or airport you will be flying from?

================================ Human Message =================================

hegeduscs is the username and London is the departure city

================================== Ai Message ==================================

Just to confirm, you would like to book a flight from London to Budapest on January 1, 2023. I will proceed with the booking. 

Please confirm so I can finalize the booking.

================================ Human Message =================================

I confirm the booking

================================== Ai Message ==================================
Tool Calls:
  mock_book_flight (call_VDqEiMGqSfjLdBgPLHqTFjFg)
 Call

Now, we need to execute the tool call ourselves, and insert it into the history. 

In [11]:
from langchain_core.messages import ToolMessage

tool_to_call = selector_response.tool_calls[0]["name"].lower()
selected_tool = next(tool for tool in all_tools if tool.name == tool_to_call)
print(selected_tool)

tool_output = selected_tool.invoke(input=selector_response.tool_calls[0]["args"])
print(tool_output)

tool_response = ToolMessage(content=tool_output, tool_call_id=selector_response.tool_calls[0]["id"])
print(tool_response)

#Be advised, adding this message to the history is not necessary here, we will invoke the model again with the tool response to get the next message
tool_chain_with_history.get_session_history("1").add_message(tool_response)

#Let's see current chat history to see what's next
history = tool_chain_with_history.get_session_history("1").get_messages()
history_prompt = ChatPromptTemplate.from_messages(history)
history_prompt.pretty_print()

name='mock_book_flight' description='This sensitive tool can be used to book flights.' args_schema=<class 'langchain_core.utils.pydantic.mock_book_flight'> func=<function mock_book_flight at 0x0000023E2E3805E0>
Flight booked from London to Budapest on 2023-01-01. Flight number is LH123 with Lufthansa.
content='Flight booked from London to Budapest on 2023-01-01. Flight number is LH123 with Lufthansa.' tool_call_id='call_VDqEiMGqSfjLdBgPLHqTFjFg'
================================ Human Message =================================

Book me a flight to Budapest on 2023-01-01.

================================== Ai Message ==================================

I need to know your departure location to book the flight for you. Could you please provide the city or airport you will be flying from?

================================ Human Message =================================

hegeduscs is the username and London is the departure city

================================== Ai Message ===============

In [12]:
selector_response = tool_chain_with_history.invoke({"input": []}, config={"configurable": {"session_id": "1"}},)
selector_response.pretty_print()

================================== Ai Message ==================================

Your flight has been successfully booked from London to Budapest on January 1, 2023. The flight number is LH123 with Lufthansa. If you need any further assistance or details, feel free to ask!


## Chapter 2 Langchain Agents 

We will do the same but with Langchain/Langgraph agents. 

We are not getting into Langgraph deeply yet, just utilize simple agents from that library. The create_react_agent allows us to be very simple.  

Scenario is the same, tools are same: travel assistant. 

**IMPORTANT**: run the tools section from above first. 

References: 

https://python.langchain.com/v0.2/docs/how_to/migrate_agent/#iterating-through-steps

https://langchain-ai.github.io/langgraph/reference/prebuilt/

In [12]:
%pip install -q langgraph

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Create the agent object, give it tools and a system prompt. The checkpointer is the chat memory (in memory). 

In [15]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import SystemMessage

AGENT_SYSTEM_PROMPT = SystemMessage(content="You are a travel assistant. You are asked to help the user with their travel plans.")

plain_tools = [mock_query_hotel_bookings, mock_query_flight_bookins, mock_query_car_rentals]
sensitive_tools = [mock_book_flight, mock_book_hotel, mock_book_car]
all_tools = plain_tools + sensitive_tools

agent = create_agent(llm, all_tools, system_prompt=AGENT_SYSTEM_PROMPT, checkpointer=MemorySaver())
config = {"configurable": {"thread_id": "1"}}

input = {"messages": [("user", "What flights do I have booked?")]}

for s in agent.stream(input, config, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

What flights do I have booked?
================================== Ai Message ==================================

Please provide your name so I can look up your flight bookings.


In [16]:
input = {"messages": [("user", "My username is hegeduscs")]}

for s in agent.stream(input, config, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

My username is hegeduscs
================================== Ai Message ==================================
Tool Calls:
  mock_query_flight_bookins (call_y68LbWzrUWhKtytVcoHteF1Z)
 Call ID: call_y68LbWzrUWhKtytVcoHteF1Z
  Args:
    user: hegeduscs
================================= Tool Message =================================
Name: mock_query_flight_bookins

Current bookings: 2 flights booked to Budapest from Berlin.
================================== Ai Message ==================================

You have 2 flights booked to Budapest from Berlin. If you need more details or assistance with anything else, feel free to ask!


Let's continue the discussion. 

In [17]:
input = {"messages": [("user", "Book me a flight to Budapest")]}

for s in agent.stream(input, config, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

Book me a flight to Budapest
================================== Ai Message ==================================

Could you please provide the following details for your flight booking to Budapest:

1. Your departure city (e.g., Berlin)
2. The date you wish to fly

With this information, I can proceed with the booking.


In [18]:
input = {"messages": [("user", "from Helsinki and on 2023-01-01")]}

for s in agent.stream(input, config, stream_mode="values"):
    message = s["messages"][-1]
    if isinstance(message, tuple):
        print(message)
    else:
        message.pretty_print()

================================ Human Message =================================

from Helsinki and on 2023-01-01
================================== Ai Message ==================================
Tool Calls:
  mock_book_flight (call_Mos1h24qapQQacPpwwan3nQg)
 Call ID: call_Mos1h24qapQQacPpwwan3nQg
  Args:
    user: hegeduscs
    origin: Helsinki
    destination: Budapest
    date: 2023-01-01
================================= Tool Message =================================
Name: mock_book_flight

Flight booked from Helsinki to Budapest on 2023-01-01. Flight number is LH123 with Lufthansa.
================================== Ai Message ==================================

Your flight from Helsinki to Budapest has been successfully booked for January 1, 2023. The flight number is LH123 with Lufthansa. If you need anything else, just let me know!


Let's print full history. 

In [19]:
state = agent.get_state(config)
chat_history = state.values["messages"]

history_prompt = ChatPromptTemplate.from_messages(chat_history)
history_prompt.pretty_print()

================================ Human Message =================================

What flights do I have booked?

================================== Ai Message ==================================

Please provide your name so I can look up your flight bookings.

================================ Human Message =================================

My username is hegeduscs

================================== Ai Message ==================================
Tool Calls:
  mock_query_flight_bookins (call_y68LbWzrUWhKtytVcoHteF1Z)
 Call ID: call_y68LbWzrUWhKtytVcoHteF1Z
  Args:
    user: hegeduscs

================================= Tool Message =================================
Name: mock_query_flight_bookins

Current bookings: 2 flights booked to Budapest from Berlin.

================================== Ai Message ==================================

You have 2 flights booked to Budapest from Berlin. If you need more details or assistance with anything else, feel free to ask!

=======================